In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers.experimental.preprocessing import Resizing, Rescaling
from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
import PIL
import os
import zipfile
import pathlib
import matplotlib.image as mpimg

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import shutil
shutil.rmtree('/root/.keras/datasets/animals', ignore_errors=True)

In [9]:
# each image is an array of individual img arrays :o
import pathlib
full_path = os.path.abspath("/content/drive/MyDrive/DataMiningProject/animals.zip")
data_dir = tf.keras.utils.get_file('animals.zip', f'file://{full_path}', extract=True)

parent_dir = pathlib.Path(data_dir).with_suffix('').parent
data_dir = parent_dir / 'animals'
print(data_dir)

/root/.keras/datasets/animals


In [10]:
# IMG_SIZE = 128

# resize_and_rescale = Sequential([
#     Resizing(IMG_SIZE, IMG_SIZE),
#     Rescaling(1./255),
# ])

data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
])

model = Sequential([
    layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    data_augmentation,
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(90)
])

In [11]:
batch_size = 50
img_height, img_width = 128, 128

training_data = tf.keras.utils.image_dataset_from_directory(data_dir, validation_split=0.15, subset="training", seed=123, image_size=(img_height, img_width), batch_size=batch_size)
testing_data = tf.keras.utils.image_dataset_from_directory(data_dir, validation_split=0.15, subset="validation", seed=123, image_size=(img_height, img_width), batch_size=batch_size)


train_ds = tf.keras.utils.image_dataset_from_directory(data_dir, validation_split=0.15, subset="validation", seed=123, image_size=(img_height, img_width), batch_size=batch_size)
class_names = train_ds.class_names
num_classes = len(class_names)


Found 5419 files belonging to 90 classes.
Using 4607 files for training.
Found 5419 files belonging to 90 classes.
Using 812 files for validation.
Found 5419 files belonging to 90 classes.
Using 812 files for validation.


In [12]:
normalization_layer = layers.Rescaling(1./255)
normalized_ds = training_data.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [13]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = training_data.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = testing_data.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=25
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/25
93/93 [==============================] - 104s 1s/step - loss: 4.4717 - accuracy: 0.0132 - val_loss: 4.3867 - val_accuracy: 0.0419
Epoch 2/25


In [ ]:
parent_dir = pathlib.Path(data_dir).with_suffix('').parent
data_dir = parent_dir / 'animals'

imgPath = f'{str(data_dir)}/crab/0a37acd141.png'

img = mpimg.imread(imgPath) #Replace "image.jpg" with the path of your image
plt.imshow(img)

In [ ]:
img = tf.keras.utils.load_img(imgPath, target_size=(img_height, img_width))
img_array = tf.expand_dims(img, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image is most likely a(n) {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)